## Data

In [1]:
# # read 'Westlaw' page from excel file
# # extract the content of the page

# import pandas as pd
# import numpy as np
# import re

# # read excel file
# # header third row
# # first two rows drop
# df = pd.read_excel('Westlaw_China_2022.xlsx', sheet_name='Westlaw', header=2)

# df.head()

,Title,Citation,Journal,Authors,Abstract,Methods,Data source,Conclusion,Unnamed: 8
0,Reputational Sanctions as A Self-regulatory To...,50 No. 4 Securities Regulation Law Journal ART 2,Securities Regulation Law Journal,Robin Hui Huang,This paper thus aims to examine more recent us...,Dataset analysis & interviewing,"SHSE & SZSE websites, CSMAR",This paper focuses on an important yet underst...,NaN
1,STABILITY JUSTICE: PETITIONERS VERSUS NON-PETI...,"Yuqing Feng & Yu Zeng, Stability Justice: Peti...",Law and Society Review,Yuqing Feng & Yu Zeng,"Different from “judicial repression,” stabilit...",Dataset analysis (regression),China Judgments Online,This article studies the mechanism of stabilit...,NaN
2,THE JUDGE AS A NEGOTIATOR: CLAIMS NEGOTIATING ...,"Xin He, The Judge As A Negotiator: Claims Nego...",Law and Social Inquiry,Xin He,I propose a three-way model of negotiation on ...,Dataset analysis & interviewing,China Law Yearbooks,This article details a Chinese judge's settlem...,NaN
3,FROM DATAFICATION TO DATA STATE: MAKING SENSE ...,"Anne S. Y. Cheung & Yongxi Chen, From Datafica...",Law and Social Inquiry,Anne S. Y. Cheung & Yongxi Chen,We live in an age of datafication wherein near...,Legal analysis,NaN,"Above, we discuss the various salient features...",NaN
4,THE LAW OF THE SEA AND PRC GRAY-ZONE OPERATION...,"Rob McLaughlin, The Law of the Sea and Prc Gra...",American Journal of International Law,Rob McLaughlin,This Current Development describes the nature ...,Legal analysis,NaN,This Current Development has called attention ...,NaN


In [6]:
import pandas as pd
import numpy as np
url= 'https://raw.githubusercontent.com/echodpp/ai_reading_tool/main/topic_model/Westlaw_China_2022.csv'
df_new = pd.read_csv(url)
df_new.head()


,Title,Abstract,Methods,Data source,Conclusion
0,Reputational Sanctions as A Self-regulatory To...,This paper thus aims to examine more recent us...,Dataset analysis & interviewing,"SHSE & SZSE websites, CSMAR",This paper focuses on an important yet underst...
1,STABILITY JUSTICE: PETITIONERS VERSUS NON-PETI...,"Different from “judicial repression,” stabilit...",Dataset analysis (regression),China Judgments Online,This article studies the mechanism of stabilit...
2,THE JUDGE AS A NEGOTIATOR: CLAIMS NEGOTIATING ...,I propose a three-way model of negotiation on ...,Dataset analysis & interviewing,China Law Yearbooks,This article details a Chinese judge's settlem...
3,FROM DATAFICATION TO DATA STATE: MAKING SENSE ...,We live in an age of datafication wherein near...,Legal analysis,NaN,"Above, we discuss the various salient features..."
4,THE LAW OF THE SEA AND PRC GRAY-ZONE OPERATION...,This Current Development describes the nature ...,Legal analysis,NaN,This Current Development has called attention ...


## One word topic

In [ ]:
# download stopwords from nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [12]:
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# 1. Data Preparation

# Concatenate relevant fields
df_new['text'] = df_new['Title'] + ' ' + df_new['Abstract'] + ' ' + df_new['Methods'] + ' ' + df_new['Data source'] + ' ' + df_new['Conclusion']
df_new = df_new.fillna("")
# Preprocess the text
# self-defined stop words
custom_stopwords = ['china', 'law','chinese','u']
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    stop_words.update(custom_stopwords)
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t.isalpha()]  # remove non-alphabetic tokens
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

df_new['tokens'] = df_new['text'].apply(preprocess)

# 2. Feature Extraction

# Create a dictionary and corpus for LDA
dictionary = corpora.Dictionary(df_new['tokens'])
corpus = [dictionary.doc2bow(text) for text in df_new['tokens']]

# 3. Topic Modeling

# Define and train the LDA model
num_topics = 8  # You can change this based on your needs
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

# Assign dominant topic to each paper
df_new['dominant_topic'] = [sorted(lda_model[doc], key=lambda x: x[1], reverse=True)[0][0] for doc in corpus]
# top 4 words for each topic
# def get_topic_words(topic_id):
#     return [(topic[0], round(topic[1], 2)) for topic in lda_model.show_topic(topic_id, topn=4)]

# df_new['topic_words'] = df_new['dominant_topic'].apply(get_topic_words)

print(df_new[['Title', 'dominant_topic']])





Topic: 0 
Words: 0.018*"education" + 0.011*"equality" + 0.010*"maoist" + 0.008*"liability" + 0.008*"educational" + 0.008*"socialist" + 0.008*"constitutional" + 0.007*"workforce" + 0.006*"petitioner" + 0.006*"reform"

Topic: 1 
Words: 0.016*"state" + 0.014*"legal" + 0.008*"international" + 0.008*"article" + 0.007*"protection" + 0.007*"security" + 0.006*"enforcement" + 0.006*"rule" + 0.006*"trade" + 0.006*"development"

Topic: 2 
Words: 0.017*"public" + 0.013*"mediation" + 0.009*"people" + 0.008*"support" + 0.008*"judge" + 0.008*"penalty" + 0.008*"death" + 0.007*"platform" + 0.007*"policy" + 0.006*"information"

Topic: 3 
Words: 0.019*"security" + 0.014*"national" + 0.009*"legal" + 0.009*"model" + 0.008*"action" + 0.008*"investment" + 0.008*"fdi" + 0.007*"issue" + 0.007*"state" + 0.007*"liberalization"

Topic: 4 
Words: 0.012*"state" + 0.012*"article" + 0.011*"legal" + 0.010*"united" + 0.009*"trade" + 0.008*"arctic" + 0.007*"trademark" + 0.007*"court" + 0.006*"wto" + 0.006*"sanction"

To

In [13]:

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.085830  0.009649       1        1  20.894480
6     -0.062916  0.031681       2        1  16.078491
5      0.075501 -0.121591       3        1  16.055109
4     -0.073000 -0.067348       4        1  12.621563
2      0.018631  0.164742       5        1  10.390813
3     -0.059688 -0.021853       6        1   8.517798
7      0.009450 -0.011119       7        1   7.872154
0      0.177851  0.015838       8        1   7.569592, topic_info=            Term       Freq      Total Category  logprob  loglift
45      security  48.000000  48.000000  Default  30.0000  30.0000
44      sanction  25.000000  25.000000  Default  29.0000  29.0000
2658   education  19.000000  19.000000  Default  28.0000  28.0000
36        public  28.000000  28.000000  Default  27.0000  27.0000
921   technology  33.000000  33.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
2661         era   3.562568  12.559813   Topic8  -5.5604   1.3210
992          law   3.552746  16.310602   Topic8  -5.5631   1.0569
125        legal   3.564917  98.389967   Topic8  -5.5597  -0.7368
892       people   3.562670  23.620477   Topic8  -5.5604   0.6894
122     judicial   3.560200  28.490791   Topic8  -5.5610   0.5013

[489 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
66        1  0.133710  action
66        3  0.133710  action
66        4  0.066855  action
66        6  0.534838  action
66        7  0.066855  action
...     ...       ...     ...
61        7  0.295916    year
61        8  0.073979    year
3002      8  0.939418   youth
422       1  0.787023    zone
422       2  0.131171    zone

[971 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 5, 3, 4, 8, 1])

## Two/Three words topic

In [18]:
import gensim
bigram = gensim.models.Phrases(df_new['tokens'], min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[df_new['tokens']], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
bigram_trigram_stopwords = [ 'written_form'] # add more as needed

def replace_similar_phrases(raw_text):
    # Replace similar phrases before tokenizing
    raw_text = raw_text.replace('forced technology transfer', 'technology transfer')
    # Add other replacement rules as needed
    return raw_text

df_new['processed_text'] = df_new['text'].apply(replace_similar_phrases)
# Assuming simple whitespace-based tokenization:
df_new['processed_tokens'] = df_new['processed_text'].apply(lambda x: x.split())


def extract_bigrams_trigrams(texts):
    bigrams = [bigram_mod[doc] for doc in texts]
    trigrams = [trigram_mod[bigram_mod[doc]] for doc in bigrams]
    
    # Filtering to keep only bigrams and trigrams
    bigram_trigram_tokens = []
    for doc in trigrams:
        bigram_trigram_tokens.append(
            [token for token in doc if "_" in token and token not in bigram_trigram_stopwords]
        )
        
    return bigram_trigram_tokens

df_new['bigram_trigram_tokens'] = extract_bigrams_trigrams(df_new['processed_tokens'])




In [19]:
dictionary = corpora.Dictionary(df_new['bigram_trigram_tokens'])
corpus = [dictionary.doc2bow(text) for text in df_new['bigram_trigram_tokens']]

# LDA with adjusted parameters
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, alpha='auto', eta='auto')

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

# Assign dominant topic to each paper
df_new['dominant_topic'] = [sorted(lda_model[doc], key=lambda x: x[1], reverse=True)[0][0] for doc in corpus]

print(df_new[['Title', 'dominant_topic']])


Topic: 0 
Words: 0.592*"national_security" + 0.244*"public_support" + 0.137*"death_penalty" + 0.003*"intellectual_property" + 0.003*"technology_transfer" + 0.003*"gray_zone" + 0.003*"homeward_trend" + 0.003*"class_action" + 0.003*"insider_trading" + 0.003*"contractual_formality"

Topic: 1 
Words: 0.091*"national_security" + 0.091*"technology_transfer" + 0.091*"intellectual_property" + 0.091*"class_action" + 0.091*"gray_zone" + 0.091*"contractual_formality" + 0.091*"insider_trading" + 0.091*"homeward_trend" + 0.091*"public_support" + 0.091*"educational_equality"

Topic: 2 
Words: 0.091*"national_security" + 0.091*"intellectual_property" + 0.091*"technology_transfer" + 0.091*"class_action" + 0.091*"educational_equality" + 0.091*"insider_trading" + 0.091*"homeward_trend" + 0.091*"contractual_formality" + 0.091*"gray_zone" + 0.091*"public_support"

Topic: 3 
Words: 0.831*"educational_equality" + 0.017*"national_security" + 0.017*"intellectual_property" + 0.017*"technology_transfer" + 0.017

In [20]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# This might take a bit of time
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.341455  0.165102       1        1  40.210678
5     -0.268063  0.248172       2        1  27.376672
6     -0.030082 -0.210829       3        1   9.444587
3     -0.017680 -0.101855       4        1   7.647423
4     -0.014878 -0.065382       5        1   6.473531
7     -0.014878 -0.065382       6        1   6.473480
1      0.002063  0.015087       7        1   1.186825
2      0.002063  0.015087       8        1   1.186804, topic_info=                    Term       Freq      Total Category  logprob  loglift
1      national_security  20.000000  20.000000  Default  11.0000  11.0000
8    technology_transfer  14.000000  14.000000  Default  10.0000  10.0000
9   educational_equality   6.000000   6.000000  Default   9.0000   9.0000
10        homeward_trend   5.000000   5.000000  Default   8.0000   8.0000
3        insider_trading   5.000000   5.000000  Default   7.0000   7.0000
..                   ...        ...        ...      ...      ...      ...
9   educational_equality   0.090629   6.061578   Topic8  -2.3979   0.2310
0              gray_zone   0.090628   6.272985   Topic8  -2.3979   0.1967
5         public_support   0.090628   8.968997   Topic8  -2.3979  -0.1609
8    technology_transfer   0.090629  14.429351   Topic8  -2.3979  -0.6363
1      national_security   0.090630  20.717517   Topic8  -2.3979  -0.9980

[99 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
6         3  0.710841           class_action
7         3  0.710842  contractual_formality
4         1  0.933869          death_penalty
9         4  0.824868   educational_equality
0         2  0.956482              gray_zone
10        5  0.784264         homeward_trend
3         6  0.784269        insider_trading
2         2  0.844073  intellectual_property
1         1  0.965367      national_security
5         1  0.891961         public_support
8         2  0.970245    technology_transfer, R=11, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 7, 4, 5, 8, 2, 3])

## One & Two/Three words topic

In [12]:
import gensim
# Include bigrams and trigrams

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t.isalpha()]  # remove non-alphabetic tokens
    tokens = [t for t in tokens if t not in stop_words]
    return tokens



bigram = gensim.models.Phrases(df_new['tokens'], min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[df_new['tokens']], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

df_new['tokens_bigrams'] = make_bigrams(df_new['tokens'])
df_new['tokens_trigrams'] = make_trigrams(df_new['tokens'])

dictionary = corpora.Dictionary(df_new['tokens_trigrams'])
corpus = [dictionary.doc2bow(text) for text in df_new['tokens_trigrams']]

# LDA with adjusted parameters
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, alpha='auto', eta='auto')


In [16]:


import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.048882 -0.007844       1        1  19.276343
4      0.062979  0.040609       2        1  17.618140
3     -0.001769  0.026860       3        1  12.672667
1      0.037884 -0.172468       4        1  12.179254
7     -0.147071  0.058157       5        1  11.935439
5     -0.090964 -0.040903       6        1  10.893178
2      0.107566  0.087254       7        1  10.719429
6     -0.017509  0.008333       8        1   4.705549, topic_info=          Term       Freq      Total Category  logprob  loglift
44    sanction  26.000000  26.000000  Default  30.0000  30.0000
2700    prison  20.000000  20.000000  Default  29.0000  29.0000
200      court  59.000000  59.000000  Default  28.0000  28.0000
2431         u  19.000000  19.000000  Default  27.0000  27.0000
1430     trade  27.000000  27.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
75     article   2.959447  86.479314   Topic8  -5.2471  -0.3185
125      legal   2.963211  97.202857   Topic8  -5.2459  -0.4341
56         two   2.384340  17.609762   Topic8  -5.4632   1.0569
250        one   2.383716  25.360084   Topic8  -5.4635   0.6919
1071        ip   2.381472  17.097366   Topic8  -5.4644   1.0852

[508 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1038      2  0.680755       act
1038      3  0.272302       act
67        1  0.068445  activity
67        2  0.136891  activity
67        3  0.410672  activity
...     ...       ...       ...
2852      5  0.245822       wto
2852      7  0.655526       wto
2853      7  0.923684       xxi
2854      7  0.658676     xxiii
3016      2  0.810051     youth

[961 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 2, 8, 6, 3, 7])

## try bertopic 

In [18]:
from bertopic import BERTopic

# Combine your text data
# df_new['text'] = df_new['Title'] + ' ' + df_new['Abstract'] + ' ' + df_new['Methods'] + ' ' + df_new['Data source'] + ' ' + df_new['Conclusion']

# Create BERTopic model
topic_model = BERTopic(language="english", calculate_probabilities=True)

# Fit to your data
topics, probabilities = topic_model.fit_transform(df_new['text'])

# To view the topics
topic_model.get_topic_info()


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Topic,Count,Name,Representation,Representative_Docs
0,-1,26,-1_the_of_and_to,"[the, of, and, to, in, that, china, for, this,...",[RETHINKING THE GLOBAL GOVERNANCE OF MIGRANT D...
1,0,11,0_the_of_in_and,"[the, of, in, and, to, as, that, law, is, china]","[FORMALITY, FREEDOM OF CONTRACT AND CHINA'S NE..."
2,1,11,1_the_and_of_to,"[the, and, of, to, us, in, trade, china, techn...",[THE POLITICAL ECONOMY OF TECHNOLOGY TRANSFER ...


In [19]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word.lower() not in stop_words])

df_new['cleaned_text'] = df_new['text'].apply(remove_stopwords)


In [20]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", nr_topics="auto", calculate_probabilities=True)
topics, probabilities = topic_model.fit_transform(df_new['cleaned_text'])


In [21]:
# If you find topics 10 and 15 to be similar, you can merge them:
topic_model.update_topics(df_new['cleaned_text'], topics, n_gram_range=(1, 2))


In [23]:
topic_info = topic_model.get_topic_info()
print(topic_info)


   Topic  Count                          Name  \
0     -1     23  -1_china_law_article_chinese   
1      0     14    0_system_china_law_chinese   
2      1     11       1_us_china_trade_chinas   

                                      Representation  \
0  [china, law, article, chinese, legal, labor, w...   
1  [system, china, law, chinese, legal, prison, m...   
2  [us, china, trade, chinas, sanctions, states, ...   

                                 Representative_Docs  
0  [FACES SECURITIES REGULATION STATE AUTHORITY: ...  
1  [ECONOMIC CHALLENGES PRISON RENOVATION REFORM ...  
2  [FITTING CHINA--US TRADE WTO TRADE LAW--NATION...  


In [31]:
# from bertopic import BERTopic

# # Define custom stopwords
# custom_stopwords = ["china", "law", "united_states", ...]  # Add more if needed

# # Create BERTopic model
# topic_model = BERTopic(language="english", calculate_probabilities=True, nr_topics='auto')

# # Update the stopwords
# topic_model.update_stopwords(custom_stopwords)

# # Fit to your data
# topics, probabilities = topic_model.fit_transform(df_new['text'])

# # To view the topics
# topic_model.get_topic_info()

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/umap/__init__.py:36: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


AttributeError: 'BERTopic' object has no attribute 'update_stopwords'